# Hash and HMAC
- Fathi Al Adha Hylmi
- Louis Widi Anandaputra

**Overview**
<br>
The hash would use keyed Bitwise-Rotated Exclusive OR methods. Meaning, the first block would be produced by a key. Meanwhile, the HMAC would use the Bitwise-Rotated Exclusive OR for both the inner hash and outer hash

**Method for finding the same values**
<br>
In finding two data which would provide the same values for Hash and HMAC value, we would take into account the type of hash function we will be using in this work. We are using the bitwise rotated exclusive OR (XOR) operations and it basically is an iterative method done on all of the data that are splitted into blocks. 

- It would take the **inner** key and initial value to be XORed to create the first block.
- The first block would then be rotated left-wise one step and XORed with the next block. This step is iterated until the last block is XORed
- The result from XOR-ing the last block would be the inner hash value

In the next step, an outer hash value is produced to a similar method. 
- It would take the inner key and **outer** value to be XORed to create the first block.
- The first block would then be rotated left-wise one step and XORed with the **inner hash value**. 
- The result from XOR-ing would be the outer hash value


Working from this intuition, to first find the hash value, the bitwise rotated XOR is basically rotating the current block to be XORed with the next block. The value of the key would not be important in two data as it is XORed with the initial value to create the first block. From here, if the current block is XORed with 0 as the next block's value, it would still be the same value. Therefore, if the current block is rotated *n* times, with *n* being the bit length for a block alongside having it XORed with 0 to stay the same value, it would produce the same value with smaller data that is not being XORed with 0 or rotated.

In this example, we will explore the use a maximum of 8 bit representation of a block. We have two values:
- 11 (2 blocks)
- 1100000000 (10 blocks) --> having 8 zeroes
As the hash value of **11** is **00110100**, the hash value of **1100000000** is also **00110100**. This is because 01001010 is being rotated 8 times while being XORed with 0 along the way for the second data  as the next blocks only consist of 0

Acting on the first evidence,to produce the result on the HMAC value, same inner hash result is needed. Therefore, we can use the same strategy for the HMAC. By using blocks with the length of 16 bits, we have two data:
- 1111(4 blocks)
- 11110000000000000000 (16 blocks)
The two data provided HMAC of **01000111010011001** and since the key (outer key) and initial value used on HMAC of the two data, it provide the same results, added by the same inner hash value of **0000111110111011**. Therefore, we acquired two data that provide the same Hash and HMAC using the same strategy.

## Defining Functions

In [1]:
from os import urandom

# Initial key generator
def initialGenKey(length):
    return bytearray(urandom(length))

def breakMessage(message, len_of_blocks):
    if isinstance(message, str):
        message = message.encode()
    list_of_blocks = []
    for i in range(0, len(message), len_of_blocks):
        block = message[i:i + len_of_blocks]
        if len(block) == len_of_blocks:
            list_of_blocks.append(block)
        else:
            c = len_of_blocks - len(block)
            block += b' ' * c
            list_of_blocks.append(block)
    return list_of_blocks
    

def bitwise_rotate_left(val, r_bits, max_bits): 
    """
    val : value to be rotated
    r_bits = number of rotation step
    max_bits = maximum bit rotations
    """
    return (val << r_bits%max_bits) & (2**max_bits-1) | ((val & (2**max_bits-1)) >> (max_bits-(r_bits%max_bits)))

def hashFunction(message, len_of_block, key, iv, hmac_outer = False):
    """
    message : plaintext (string if not for outer hash in hmac, block array for outer hash in hmac)
    len_of_block: n bytes of block length
    key : key for XOR with Initial Value
    IV: Initial Value for XOR with key
    """
    blockList = message
    if hmac_outer == False:
        blockList = breakMessage(message, len_of_block)
        blockList = [int.from_bytes(i, byteorder = 'big') for i in blockList]
    
    # Start of Rotated XOR
    block = iv^key
    for m in blockList:
        
        temp = bitwise_rotate_left(block, 1, (8*(len_of_block)))
        block = m ^ temp 
    
    return block

def HMAC(message, key, len_of_block, iv):
    """
    message : plaintext (string if not for outer hash in hmac, block array for outer hash in hmac)
    len_of_block: n bytes of block length
    key : key for operation with outer and inner padding in hash function
    IV: Initial Value for XOR with key in hash function
    """
    key = int.from_bytes(key, byteorder = 'big')
    iv = int.from_bytes(iv, byteorder = 'big')

        
    o_key_pad = key ^ 0x5C
    i_key_pad = key ^ 0x36

    inner_hash = hashFunction(message, len_of_block, i_key_pad, iv) 
    print("inner_hash:",(inner_hash))
    inner_hash = [inner_hash]
    hmac_output = hashFunction(inner_hash, len_of_block, o_key_pad, iv, hmac_outer = True)

    return hmac_output

# This function is explicitly used to compare the two data to acquire the same values
def HMAC_Compare(message, key, len_of_block, iv):
    """
    message : plaintext (string if not for outer hash in hmac, block array for outer hash in hmac)
    len_of_block: n bytes of block length
    key : key for operation with outer and inner padding in hash function
    IV: Initial Value for XOR with key in hash function
    """
    key = int.from_bytes(key, byteorder = 'big')
    iv = int.from_bytes(iv, byteorder = 'big')

        
    o_key_pad = key ^ 0x5C
    i_key_pad = key ^ 0x36

    inner_hash = hashFunction(message, len_of_block, i_key_pad, iv, hmac_outer = True) # NOTE for testing, make the hmac_outer = True
    print("inner_hash:",bin(inner_hash))
    inner_hash = [inner_hash]
    hmac_output = hashFunction(inner_hash, len_of_block, o_key_pad, iv, hmac_outer = True)

    return hmac_output




## Demo

In [14]:
# Initialization
text = "Hello"
len_of_block_hash = 2

# Generate key and IV
key = initialGenKey(len_of_block_hash)
iv = initialGenKey(len_of_block_hash)
print(key, iv)

bytearray(b'v\x85') bytearray(b'\xd8\xd2')


In [15]:
# Using saved key and iv
key = bytearray(b'\x8f\xf7') 
iv = bytearray(b'\xcf:')

HMAC_VALUE = HMAC(text, key, len_of_block_hash, iv)
print("HMAC: ", HMAC_VALUE)

inner_hash: 37303
HMAC:  4245


The HMAC Value is shown as an integer representation, but it can be shown into a binary representation

## Finding Same Value for Hash and HMAC

### Hash

In [16]:
bin(hashFunction([0b1, 0b1], 
                 1, 
                 int.from_bytes(key, byteorder = 'big'), 
                 int.from_bytes(iv, byteorder = 'big'), 
                 hmac_outer = True))

'0b110100'

In [17]:
bin(hashFunction([0b1, 0b1, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0], 
                 1, 
                 int.from_bytes(key, byteorder = 'big'), 
                 int.from_bytes(iv, byteorder = 'big'), 
                 hmac_outer = True))

'0b110100'

### HMAC

In [18]:
data1 = [0b1, 0b1, 0b1, 0b1, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0, 0b0]
data2 = [0b1,0b1, 0b1, 0b1]
bin(HMAC_Compare(data1, key, len_of_block_hash, iv))


inner_hash: 0b111110111011


'0b1000111010011001'

In [19]:
bin(HMAC_Compare(data2, key, len_of_block_hash, iv))

inner_hash: 0b111110111011


'0b1000111010011001'